In [ ]:
!pip install group_lasso
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV
from group_lasso import GroupLasso
import matplotlib.pyplot as plt

In [ ]:
# Data loading
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/DSP2025 Homework 5/dataset.csv"
df = pd.read_csv(file_path)
print("Data preview:")
print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data preview:
      Age    LWT  Race  Smoker  PTL  Hypertension     UI  FTV     BWT
0    19.0  182.0     2   False    0         False   True    0  2523.0
1    33.0  155.0     3   False    0         False  False    3  2551.0
2    20.0  105.0     1    True    0         False  False    1  2557.0
3    21.0  108.0     1    True    0         False   True    2  2594.0
4    18.0  107.0     1    True    0         False   True    0  2600.0
..    ...    ...   ...     ...  ...           ...    ...  ...     ...
184  28.0   95.0     1    True    0         False  False    2  2466.0
185  14.0  100.0     3   False    0         False  False    2  2495.0
186  23.0   94.0     3    True    0         False  False    0  2495.0
187  17.0  142.0     2   False    0          True  False    0  2495.0
188  21.0  130.0     1    True    0          True  False    3  2495.0

[189 rows x 9 co

In [ ]:
# Feature construction
#(a) Create third-order polynomial features for Age and LWT.

X_poly_input = df[['Age', 'LWT']]

# 建立 PolynomialFeatures：degree=3 表示三次多項式，include_bias=False 不加常數項
poly = PolynomialFeatures(degree=3, include_bias=False)

# 轉換資料：會產生 Age, LWT, Age^2, Age*LWT, ..., LWT^3 共 9 項
X_poly = poly.fit_transform(X_poly_input)

# 獲得每個產生的新特徵名稱
poly_feature_names = poly.get_feature_names_out(['Age', 'LWT'])

# 將轉換後的結果轉回 DataFrame，方便後續合併
df_poly = pd.DataFrame(X_poly, columns=poly_feature_names)

print("Third-order polynomial features for Age and LWT:")
print(df_poly.head())

Third-order polynomial features for Age and LWT:
    Age    LWT   Age^2  Age LWT    LWT^2    Age^3  Age^2 LWT  Age LWT^2  \
0  19.0  182.0   361.0   3458.0  33124.0   6859.0    65702.0   629356.0   
1  33.0  155.0  1089.0   5115.0  24025.0  35937.0   168795.0   792825.0   
2  20.0  105.0   400.0   2100.0  11025.0   8000.0    42000.0   220500.0   
3  21.0  108.0   441.0   2268.0  11664.0   9261.0    47628.0   244944.0   
4  18.0  107.0   324.0   1926.0  11449.0   5832.0    34668.0   206082.0   

       LWT^3  
0  6028568.0  
1  3723875.0  
2  1157625.0  
3  1259712.0  
4  1225043.0  


In [ ]:
#(b) One-hot encode all six categorical predictors, retaining every dummy column.

# 將六個類別欄位轉為字串類型，確保能正確 one-hot 編碼
categorical_cols = ['Race', 'Smoker', 'PTL', 'Hypertension', 'UI', 'FTV']
df[categorical_cols] = df[categorical_cols].astype(str)

# 做 one-hot encoding
df_cat = pd.get_dummies(df[categorical_cols], drop_first=False).astype(float)

# 顯示全部欄位
pd.set_option('display.max_columns', None)

print("One-hot encoded categorical features (float values):")
print(df_cat.head())

One-hot encoded categorical features (float values):
   Race_1  Race_2  Race_3  Smoker_False  Smoker_True  PTL_0  PTL_1  PTL_2  \
0     0.0     1.0     0.0           1.0          0.0    1.0    0.0    0.0   
1     0.0     0.0     1.0           1.0          0.0    1.0    0.0    0.0   
2     1.0     0.0     0.0           0.0          1.0    1.0    0.0    0.0   
3     1.0     0.0     0.0           0.0          1.0    1.0    0.0    0.0   
4     1.0     0.0     0.0           0.0          1.0    1.0    0.0    0.0   

   PTL_3  Hypertension_False  Hypertension_True  UI_False  UI_True  FTV_0  \
0    0.0                 1.0                0.0       0.0      1.0    1.0   
1    0.0                 1.0                0.0       1.0      0.0    0.0   
2    0.0                 1.0                0.0       1.0      0.0    0.0   
3    0.0                 1.0                0.0       0.0      1.0    0.0   
4    0.0                 1.0                0.0       0.0      1.0    1.0   

   FTV_1  FTV_2  FTV_

In [ ]:
# Standardization
# 合併 polynomial 特徵與 one-hot 特徵
X = pd.concat([df_poly, df_cat], axis=1)

# 實例化標準化工具
scaler = StandardScaler()

# 套用標準化，轉換為 NumPy array
X_scaled = scaler.fit_transform(X)

# 如果需要保留成 DataFrame（保留欄位名）
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 檢查結果
print("Standardized feature matrix (mean ≈ 0, std ≈ 1):")
print(X_scaled_df.head())

Standardized feature matrix (mean ≈ 0, std ≈ 1):
        Age       LWT     Age^2   Age LWT     LWT^2     Age^3  Age^2 LWT  \
0 -0.801965  1.711081 -0.770373  0.371196  1.645633 -0.704032  -0.215120   
1  1.847222  0.825788  1.939789  1.862901  0.669642  1.940067   2.123102   
2 -0.612737 -0.813644 -0.625186 -0.851336 -0.724784 -0.600279  -0.752698   
3 -0.423509 -0.715278 -0.472553 -0.700095 -0.656242 -0.485615  -0.625051   
4 -0.991192 -0.748067 -0.908114 -1.007978 -0.679304 -0.797418  -0.918993   

   Age LWT^2     LWT^3    Race_1    Race_2    Race_3  Smoker_False  \
0   0.787863  1.494919 -1.016001  2.503843 -0.741067      0.802171   
1   1.407182  0.493150 -1.016001 -0.399386  1.349406      0.802171   
2  -0.761130 -0.622309  0.984251 -0.399386 -0.741067     -1.246617   
3  -0.668522 -0.577935  0.984251 -0.399386 -0.741067     -1.246617   
4  -0.815755 -0.593004  0.984251 -0.399386 -0.741067     -1.246617   

   Smoker_True     PTL_0     PTL_1     PTL_2     PTL_3  Hypertension_Fals

In [ ]:
# Elastic Net
# (a) Use ElasticNetCV to tune alpha and l1_ratio with 3-fold Cross Validation (CV).

# y 是預測目標：出生體重
y = df['BWT']

# 使用 ElasticNetCV，自動幫我們選最佳 alpha 與 l1_ratio
elastic_net_cv = ElasticNetCV(
    cv=3,
    l1_ratio=[0.1, 0.5, 0.9],
    alphas=np.logspace(-2, 1, 20) ,
    max_iter=10000,       # 加大迭代上限
    random_state=0
)
# 套用模型訓練
elastic_net_cv.fit(X_scaled, y)

# 取得最佳參數
best_alpha = elastic_net_cv.alpha_
best_l1_ratio = elastic_net_cv.l1_ratio_

# 計算 CV 中的平均 MSE（預設是 negative MSE，所以取負號）
cv_mse = -np.mean(elastic_net_cv.mse_path_, axis=1).min()

# 取得非零係數特徵名稱
nonzero_features = X.columns[elastic_net_cv.coef_ != 0]

# 顯示結果
print(f"Best alpha: {best_alpha}")
print(f"Best l1_ratio: {best_l1_ratio}")
print(f"3-fold CV mean squared error (MSE): {cv_mse:.4f}")
print("Non-zero coefficient features:")
print(nonzero_features.tolist())

Best alpha: 0.01
Best l1_ratio: 0.9
3-fold CV mean squared error (MSE): -201544.6922
Non-zero coefficient features:
['Age', 'LWT', 'Age^2', 'Age LWT', 'LWT^2', 'Age^3', 'Age^2 LWT', 'Age LWT^2', 'LWT^3', 'Race_1', 'Race_2', 'Race_3', 'Smoker_False', 'Smoker_True', 'PTL_0', 'PTL_1', 'PTL_2', 'PTL_3', 'Hypertension_False', 'Hypertension_True', 'UI_False', 'UI_True', 'FTV_0', 'FTV_1', 'FTV_2', 'FTV_3', 'FTV_4', 'FTV_6']


In [ ]:
"""
(b) Report the best (alpha, l1_ratio), the 3-fold CV mean squared error (MSE), and list
all features with non-zero coefficients.

1. Best (alpha, l1_ratio)
The best hyperparameters selected by ElasticNetCV (with 3-fold CV) are:
- alpha = 0.01
- l1_ratio = 0.9

2. 3-fold CV Mean Squared Error (MSE)
- The 3-fold cross-validation mean squared error (MSE) is approximately 201544.69.

3. Features with non-zero coefficients
- The following features have non-zero coefficients in the final model:
- Numerical (polynomial) features:
  ['Age', 'LWT', 'Age^2', 'Age LWT', 'LWT^2', 'Age^3', 'Age^2 LWT', 'Age LWT^2', 'LWT^3']

- Categorical (one-hot encoded) features:
  ['Race_1', 'Race_2', 'Race_3',
   'Smoker_False', 'Smoker_True',
   'PTL_0', 'PTL_1', 'PTL_2', 'PTL_3',
   'Hypertension_False', 'Hypertension_True',
   'UI_False', 'UI_True',
   'FTV_0', 'FTV_1', 'FTV_2', 'FTV_3', 'FTV_4', 'FTV_6']
"""

In [ ]:
# Group Lasso
# Import and use the Group Lasso implementation from the group_lasso package
# (a) Define feature groups
groups = []
for col in X.columns:
    if 'Age' in col and 'LWT' not in col:
        groups.append(0)  # Group 0: Age
    elif 'LWT' in col and 'Age' not in col:
        groups.append(1)  # Group 1: LWT
    elif 'Age' in col and 'LWT' in col:
        groups.append(2)  # Group 2: Age * LWT
    elif col.startswith('Race_'):
        groups.append(3)  # Group 3: Race
    elif col.startswith('Smoker_'):
        groups.append(4)  # Group 4: Smoker
    elif col.startswith('PTL_'):
        groups.append(5)  # Group 5: PTL
    elif col.startswith('Hypertension_'):
        groups.append(6)  # Group 6: Hypertension
    elif col.startswith('UI_'):
        groups.append(7)  # Group 7: UI
    elif col.startswith('FTV_'):
        groups.append(8)  # Group 8: FTV
    else:
        raise ValueError(f"無法分類的特徵: {col}")

In [ ]:
# (b) Use a 3-fold CV to tune the group_reg and l1_reg.

# 確保 X_scaled 是 numpy array
X_array = np.array(X_scaled)
y_array = np.array(y)

# 預定義參數範圍
group_regs = [0.01, 0.1, 1.0]
l1_regs = [0.01, 0.1, 1.0]

best_score = float('inf')
best_params = None
best_model = None

# 使用 3-fold CV
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# 搜尋最佳參數組合
for gr in group_regs:
    for l1 in l1_regs:
        cv_scores = []
        for train_index, val_index in kf.split(X_array):
            X_train, X_val = X_array[train_index], X_array[val_index]
            y_train, y_val = y_array[train_index], y_array[val_index]

            model = GroupLasso(
                groups=groups,
                group_reg=gr,
                l1_reg=l1,
                n_iter=10000,
                tol=1e-3,
                scale_reg='group_size',
                subsampling_scheme=None,
                supress_warning=True,
                fit_intercept=True
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            mse = mean_squared_error(y_val, y_pred)
            cv_scores.append(mse)

        mean_cv_mse = np.mean(cv_scores)

        if mean_cv_mse < best_score:
            best_score = mean_cv_mse
            best_params = (gr, l1)
            best_model = model


print(f"Best group_reg: {best_params[0]}")
print(f"Best l1_reg: {best_params[1]}")
print(f"3-fold CV MSE: {best_score:.4f}")

# 保證是 1D
coef = best_model.coef_.flatten()

# 找出非零係數的index（傳回的是整數index陣列）
nonzero_idx = np.where(coef != 0)[0]

# 將 nonzero_idx 轉換為 list 用於index Python list
nonzero_idx_list = nonzero_idx.tolist()

# 用這些index去查對應的 group 編號，並找出唯一群組
selected_groups = np.unique([groups[i] for i in nonzero_idx_list])

print("Selected groups (with non-zero coefficients):", selected_groups.tolist())

Best group_reg: 1.0
Best l1_reg: 1.0
3-fold CV MSE: 488706.1925
Selected groups (with non-zero coefficients): [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
# (c) Report the best (group_reg,  l1_reg), its CV MSE, and the list of groups with nonzero coefficients.
"""
The best hyperparameter combination found using 3-fold cross-validation was:
- group_reg = 1.0
- l1_reg = 1.0
- Cross-validated Mean Squared Error (MSE) = 488,719.13

Under this configuration, the Group Lasso model retained all 9 groups, each having at least one feature with a non-zero coefficient. The selected groups are:
- Group0: Age
- Group1: LWT
- Group2: Age * LWT interactions
- Group3: Race (categorical)
- Group4: Smoker status
- Group5: PTL (previous preterm labors)
- Group6: Hypertension
- Group7: UI (urinary infection)
- Group8: FTV (number of physician visits)

This result suggests that all predefined groups contributed to predicting birth weight under the current group/l1 regularization strengths.


In [ ]:
# Results summary
"""
Results Summary Table – Elastic Net vs. Group Lasso

Item:
1. Best alpha:
   - Elastic Net: 0.01（Normalized strength, smaller and weaker）
   - Group Lasso: Not applicable

2. Best l1_ratio:
   - Elastic Net: 0.9（90% L1, biased Lasso）
   - Group Lasso: Not applicable

3. Best group_reg:
   - Elastic Net: Not applicable
   - Group Lasso: 1.0（Group Hierarchy Normalization）

4. Best l1_reg:
   - Elastic Net: Not applicable
   - Group Lasso: 1.0（Single variable L1 regularization）

5. 3-fold CV MSE:
   - Elastic Net: 201,544.69（Better）
   - Group Lasso: 488,706.19（not as good as）

6. Feature selection granularity:
   - Elastic Net: Individual feature-level (28 non-zero coefficients)
   - Group Lasso: Group-level (retained all 9 groups)

7. Regularization method:
   - Elastic Net: Mixed L1 + L2 (Elastic Net)
   - Group Lasso: Group-wise + L1 sparsity (Group Lasso)

8. Model flexibility:
   - Elastic Net: More flexible (fine-grained selection across many features)
   - Group Lasso: More rigid (works at group level; less granular control)

Summary:

Elastic Net achieved significantly lower mean squared error (MSE = 201,544.69) compared to Group Lasso (MSE = 488,706.19), and selected 28 individual predictive features.
In contrast, Group Lasso retained all 9 defined feature groups despite the strong regularization parameters, indicating that its group sparsity structure may not have aligned well with the dataset.
Overall, Elastic Net provided more accurate and interpretable results for this prediction task.
"""

'\nResults Summary Table – Elastic Net vs. Group Lasso\n\nItem:\n1. Best alpha:\n   - Elastic Net: 0.01（Normalized strength, smaller and weaker）\n   - Group Lasso: Not applicable\n\n2. Best l1_ratio:\n   - Elastic Net: 0.9（90% L1, biased Lasso）\n   - Group Lasso: Not applicable\n\n3. Best group_reg:\n   - Elastic Net: Not applicable\n   - Group Lasso: 1.0（Group Hierarchy Normalization）\n\n4. Best l1_reg:\n   - Elastic Net: Not applicable\n   - Group Lasso: 1.0（Single variable L1 regularization）\n\n5. 3-fold CV MSE:\n   - Elastic Net: 201,544.69（Better）\n   - Group Lasso: 488,706.19（not as good as）\n\n6. Feature selection granularity:\n   - Elastic Net: Individual feature-level (28 non-zero coefficients)\n   - Group Lasso: Group-level (retained all 9 groups)\n\n7. Regularization method:\n   - Elastic Net: Mixed L1 + L2 (Elastic Net)\n   - Group Lasso: Group-wise + L1 sparsity (Group Lasso)\n\n8. Model flexibility:\n   - Elastic Net: More flexible (fine-grained selection across many feat